In [26]:
# import
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import seaborn as sns
import scipy as sp
from scipy.stats import norm
from scipy import linalg
from scipy.stats import halfnorm
from ipywidgets import interactive
from IPython.display import clear_output
import timeit
#%matplotlib inline

In [27]:
#set figure defaults for IPython notebook 
#matplotlib.rcParams.update({'font.size': 18, 'lines.linewidth':4})

In [ ]:
from ipynb.fs.full.FETemp import FE

In [28]:
def plot_distribution(start_t,end_t,step_t,t_out,X_true,obs,Sample,pred):
    plt.figure(figsize=(10,5))
    plt.plot(list(np.arange(start_t,end_t+step_t,step_t)),X_true, color='red', linewidth = 2, label = 'True X')
    plt.scatter(list(np.arange(start_t,end_t+step_t,step_t)),obs, color='blue', label = 'Observations')
    plt.plot(list(np.arange(start_t,end_t+step_t,step_t)),pred[1:], color='green', linewidth = 2, label = 'Predictions')
    print('Average error between true X and observations is:', round(np.sum(abs(obs-X_true))/np.sum(abs(X_true))*100,2))
    print('Average error between true X and predicted values is:', round(np.sum(abs(pred[1:]-X_true))/np.sum(abs(X_true))*100,2))
    plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
    
    plt.figure(figsize=(10,5))
    histogram = plt.hist(Sample[:,int((t_out-start_t)/step_t+1)], bins=int(N/100), label = "Distribution at time {}".format(t_out))
    x_true_t = X_true[int((t_out-start_t)/step_t+1)] # true value at time t_out
    obs_t = obs[int((t_out-start_t)/step_t+1)] # true value at time t_out
    pred_t = pred[int((t_out-start_t)/step_t+1)] # prediction value at time t_out
    plt.plot([x_true_t,x_true_t],[0,100], color='red', linewidth = 3, label = "True value at time {}".format(t_out))
    plt.plot([obs_t,obs_t],[0,100], color='black', linewidth = 3, label = "Observation at time {}".format(t_out))
    plt.plot([pred_t,pred_t],[0,100], color='green', linewidth = 3, label = "Distribution mean at time {}".format(t_out))
    plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)

In [34]:
# T_true is an array with shape (number_node,number_of time steps) coming from FE solution for the entire time
# sensor_loc is a list of sensor locations with size (number_sensors)

def Temp_bootstrap(sensor_loc_typ,sensor_loc_list,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                   muT_sigma,mualpha_sigma,mualpha_dot_sigma,
                   N,t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                   Coords_start,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma,
                   material_dict,
                   Analysis,cri,Element_type,heat_gen,T_true,alpha_true,alpha_dot_true):

    Length = Length_c + Length_t
    num_el = num_el_c + num_el_t
    # material properties sampling
    # sampling
    k_c = np.random.normal(material_dict['k_c_mean'],material_dict['k_c_sigma'],N)
    rho_c = np.random.normal(material_dict['rho_c_mean'],material_dict['rho_c_sigma'],N)
    Cp_c = np.random.normal(material_dict['Cp_c_mean'],material_dict['Cp_c_sigma'],N)
    rho_r = np.random.normal(material_dict['rho_r_mean'],material_dict['rho_r_sigma'],N)
    H_r = np.random.normal(material_dict['H_r_mean'],material_dict['H_r_sigma'],N)
    nu_r = np.random.normal(material_dict['nu_r_mean'],material_dict['nu_r_sigma'],N)
    h_c = np.random.normal(material_dict['h_c_mean'],material_dict['h_c_sigma'],N)
    
    k_t = np.random.normal(material_dict['k_t_mean'],material_dict['k_t_sigma'],N)
    rho_t = np.random.normal(material_dict['rho_t_mean'],material_dict['rho_t_sigma'],N)
    Cp_t = np.random.normal(material_dict['Cp_t_mean'],material_dict['Cp_t_sigma'],N)
    h_t = np.random.normal(material_dict['h_t_mean'],material_dict['h_t_sigma'],N)
    
    # particles in FE
    A1 = np.random.normal(material_dict['A1_mean'],material_dict['A1_sigma'],N)
    A2 = np.random.normal(material_dict['A2_mean'],material_dict['A2_sigma'],N)
    A3 = np.random.normal(material_dict['A3_mean'],material_dict['A3_sigma'],N)
    dE1 = np.random.normal(material_dict['dE1_mean'],material_dict['dE1_sigma'],N)
    dE2 = np.random.normal(material_dict['dE2_mean'],material_dict['dE2_sigma'],N)
    dE3 = np.random.normal(material_dict['dE3_mean'],material_dict['dE3_sigma'],N)
    BB = np.random.normal(material_dict['BB_mean'],material_dict['BB_sigma'],N)
    a_c =  k_c/(rho_c*Cp_c)
    b_c =  rho_r*H_r*nu_r/(rho_c*Cp_c)
    Ch_c = h_c/k_c*a_c 
    a_t =  k_t/(rho_t*Cp_t)
    b_t =  np.zeros(N,)
    Ch_t = h_t/k_t*a_t
       
    
    n = int(int(t_end-t_start)/delt + 1) # number of states
    
    if sensor_loc_typ == "node":
        sensor_loc_n = sensor_loc_list # a list, node numbers
        sensor_loc = [(i-1) * (Length/num_el) for i in sensor_loc_n] 
    elif sensor_loc_typ == "loc":
        sensor_loc = sensor_loc_list # a list, location of sensor (m)
        sensor_loc_n = [int(round(x /  (Length/num_el))) + 1 for x in sensor_loc] # sensor location node number

    # Generating fake observations from T_true
    # observations is an array with shape (number_sensors,number_timestep)
    observations = np.zeros((len(sensor_loc_n),n)) # n is the number of time steps
    for sens in range(len(sensor_loc_n)): # observations if we put the sensor at i location
        observations[sens,:]  = T_true[sensor_loc_n[sens]-1,:] + np.random.normal(0,obs_sigma,n) 
        
    # initialization, t=0
    T_0_allp = np.ones((1,N)) 
    for node in range(0,num_el+1):
        muT_mean = T_true[node,0]
        T_0 = np.random.normal(muT_mean,muT_sigma,N) # N samples from mu ~ Normal(mu_mean,mu_sigma)
        T_0_allp = np.append(T_0_allp,T_0.reshape(1,N), axis=0)  
    T_old_allp = T_0_allp[1:,:]
    T_all_ave =np.mean(T_old_allp,axis=1).reshape(num_el+1,1) #np.zeros((num_el+1,1))
    T_all_var = np.zeros((num_el+1,1))
    T_all_var.fill(muT_sigma)
    
    alpha_0_allp = np.ones((1,N)) 
    alpha_dot_0_allp = np.ones((1,N)) 
    for el in range(0,num_el):
        mualpha_mean = alpha_true[el,0]
        alpha_0 = np.random.normal(mualpha_mean,mualpha_sigma,N) # N samples from mu ~ Normal(mu_mean,mu_sigma)
        alpha_0_allp = np.append(alpha_0_allp,alpha_0.reshape(1,N), axis=0) 
        mualpha_dot_mean = alpha_dot_true[el,0]
        #alpha_dot_0 = np.random.normal(mualpha_dot_mean,mualpha_dot_sigma,N) # N samples from mu ~ Normal(mu_mean,mu_sigma)
        alpha_dot_0 = halfnorm.rvs(loc = mualpha_dot_mean, scale = mualpha_dot_sigma, size = N)
        alpha_dot_0_allp = np.append(alpha_dot_0_allp,alpha_dot_0.reshape(1,N), axis=0) 
    alpha_old_allp = alpha_0_allp[1:,:]
    alpha_all_ave =np.mean(alpha_old_allp,axis=1).reshape(num_el,1)  # np.zeros((num_el,1))
    alpha_all_var =np.zeros((num_el,1))
    alpha_all_var.fill(mualpha_sigma)
    alpha_dot_old_allp = alpha_dot_0_allp[1:,:]
    alpha_dot_all_ave = np.mean(alpha_dot_old_allp,axis=1).reshape(num_el,1) # np.zeros((num_el,1))
    alpha_dot_all_var =np.zeros((num_el,1))
    alpha_dot_all_var.fill(mualpha_dot_sigma)
    
    for t in np.arange(t_start,t_end,delt):
        # Solve one step of FE for each particle to obtain new T_mean
        T_mean_allp = np.zeros((num_el+1,1))
        alpha_mean_allp = np.zeros((num_el,1))
        alpha_dot_mean_allp = np.zeros((num_el,1))
        for p in range(0,N):
            T_mean, Coords, alpha_mean, alpha_dot_mean = FE(t,t+delt,delt,Length_c,Length_t,num_el_c,num_el_t,
                                                            Coords_start,
                                                            air_temp_type,T_start,T_hold,
                                                            T_const,T_rate,th1,th2,T_air_sigma,
                                                            a_c[p],b_c[p],Ch_c[p],a_t[p],b_t[p],Ch_t[p],
                                                            BB[p],A1[p],A2[p],A3[p],dE1[p],dE2[p],dE3[p],
                                                            Analysis,cri,
                                                            Element_type,heat_gen,
                                                            T_old_allp[:,p].reshape(num_el+1,1),
                                                            alpha_old_allp[:,p].reshape(num_el,1),
                                                            alpha_dot_old_allp[:,p].reshape(num_el,1))
            
            T_mean_allp = np.append(T_mean_allp,T_mean[:,1].reshape(num_el+1,1),axis=1)
            alpha_mean_allp = np.append(alpha_mean_allp,alpha_mean[:,1].reshape(num_el,1),axis=1)
            alpha_dot_mean_allp = np.append(alpha_dot_mean_allp,alpha_dot_mean[:,1].reshape(num_el,1),axis=1)
            
        T_mean_allp = T_mean_allp[:,1:]
        alpha_mean_allp = alpha_mean_allp[:,1:]
        alpha_dot_mean_allp = alpha_dot_mean_allp[:,1:]

        # Sampling the new particles for each node/element
        T_new_allp = np.zeros((1,N))
        for node in range(0,num_el+1):
            T_new_node = np.random.normal(T_mean_allp[node,:],T_sigma,N)
            T_new_allp = np.append(T_new_allp,T_new_node.reshape(1,N), axis=0)
        alpha_new_allp = np.zeros((1,N))
        alpha_dot_new_allp = np.zeros((1,N))
        for el in range(0,num_el):
            alpha_new_el =  np.random.normal(alpha_mean_allp[el,:],alpha_sigma,N) # alpha_mean_allp[el,:]
            alpha_new_allp = np.append(alpha_new_allp,alpha_new_el.reshape(1,N), axis=0)
            #alpha_dot_new_el = np.random.halfnormal(alpha_dot_mean_allp[el,:],alpha_dot_sigma,N) # alpha_dot_mean_allp[el,:] 
            alpha_dot_new_el = halfnorm.rvs(loc = alpha_dot_mean_allp[el,:], scale = alpha_dot_sigma, size = N)
            alpha_dot_new_allp = np.append(alpha_dot_new_allp,alpha_dot_new_el.reshape(1,N), axis=0)
             
        
        # weight calculations
        Weight_allp = np.zeros((1,N))
        for sens in range(len(sensor_loc_n)): # len(sensor_loc_n) = number of srensors
            tn = int((t-t_start)/delt) # time step number
            weight = sp.stats.norm.pdf(observations[sens,tn], T_new_allp[sensor_loc_n[sens],:], obs_sigma) # sp.stats.norm.pdf(observation[node], T_new_allp[node,:], obs_sigma) 
            Weight = weight / sum(weight) # normalizing the weights
            Weight_allp = np.append(Weight_allp,Weight.reshape(1,N), axis=0)
        Weight_allp = Weight_allp[1:,:]

        # Resampling
        s = 0
        for i in range(len(sensor_loc_n)):
            T_new_allp[sensor_loc_n[i],:] = np.random.choice(T_new_allp[sensor_loc_n[i],:], N, p=Weight_allp[s,:])
            s +=1

        # updating results
        T_old_allp = T_new_allp[1:,:]
        T_old_ave = np.mean(T_old_allp,axis=1)
        T_old_var = np.var(T_old_allp, axis=1) 
        T_all_ave = np.append(T_all_ave,T_old_ave.reshape(num_el+1,1), axis=1)
        T_all_var = np.append(T_all_var,T_old_var.reshape(num_el+1,1), axis=1)
        
        alpha_old_allp = alpha_new_allp[1:,:]
        alpha_old_ave = np.mean(alpha_old_allp,axis=1)
        alpha_old_var = np.var(alpha_old_allp, axis=1) 
        alpha_all_ave = np.append(alpha_all_ave,alpha_old_ave.reshape(num_el,1), axis=1)
        alpha_all_var = np.append(alpha_all_var,alpha_old_var.reshape(num_el,1), axis=1)
        
        alpha_dot_old_allp = alpha_dot_new_allp[1:,:]
        alpha_dot_old_ave = np.mean(alpha_dot_old_allp,axis=1)
        alpha_dot_old_var = np.var(alpha_dot_old_allp, axis=1) 
        alpha_dot_all_ave = np.append(alpha_dot_all_ave,alpha_dot_old_ave.reshape(num_el,1), axis=1)
        alpha_dot_all_var = np.append(alpha_dot_all_var,alpha_dot_old_var.reshape(num_el,1), axis=1)
        
        if int((t-t_start)/delt)%5 == 0:
            clear_output(wait=True)
            print ("progress is : {}%".format(round((t-t_start)/(t_end-t_start)*100,1)))
        
    #T_all_ave = T_all_ave[:,1:]
    #T_all_var = T_all_var[:,1:]
    
    #alpha_all_ave = alpha_all_ave[:,1:]
    #alpha_all_var = alpha_all_var[:,1:]
    
    #alpha_dot_all_ave = alpha_dot_all_ave[:,1:]
    #alpha_dot_all_var = alpha_dot_all_var[:,1:]

    return T_all_ave, T_all_var, Coords, alpha_all_ave, alpha_all_var, alpha_dot_all_ave, alpha_dot_all_var,